In [21]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)
accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [22]:
from datasets import load_dataset
train_dataset = load_dataset('wikitext', 'wikitext-103-raw-v1', split='train')
eval_dataset = load_dataset('wikitext', 'wikitext-103-raw-v1', split='validation')
test_dataset = load_dataset('wikitext', 'wikitext-103-raw-v1', split='test')

In [23]:
print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['text'],
    num_rows: 1801350
})
Dataset({
    features: ['text'],
    num_rows: 3760
})
Dataset({
    features: ['text'],
    num_rows: 4358
})


In [24]:
train_dataset[1]

{'text': ' = Valkyria Chronicles III = \n'}

In [25]:
train_dataset[100]

{'text': ' 96 ammunition packing boxes \n'}

In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel, AutoModelWithLMHead


base_model_id = "distilbert/distilgpt2"
model = AutoModelForCausalLM.from_pretrained(base_model_id, device_map="auto")

In [27]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token

In [28]:
tokenizer("test")

{'input_ids': [9288], 'attention_mask': [1]}

In [29]:
model_inputs = tokenizer("Transformers are", return_tensors='pt').to("cuda")

output = model.generate(**model_inputs, max_new_tokens=100, pad_token_id=2, eos_token_id=tokenizer.eos_token_id)
output

tensor([[41762,   364,   389,   257,  1049,   835,   284,   651,   534,  2832,
           319,   257,   649,   983,    13,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198,   198,   198,   198,   198,   198,   198,   198,
           198,   198,   198]], device='cuda:0')

In [30]:
print(tokenizer.decode(output[0], skip_special_tokens=True))

Transformers are a great way to get your hands on a new game.


























































































In [31]:
def tokenize_func(examples):
	return tokenizer(examples["text"], truncation=True)  # max_length=512,  padding=True

encoded_train_dataset = train_dataset.map(tokenize_func, batched=True)
encoded_eval_dataset = eval_dataset.map(tokenize_func, batched=True)

# LoRA

In [32]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


## Run training

In [33]:
# if torch.cuda.device_count() > 1: # If more than 1 GPU
#     model.is_parallelizable = True
#     model.model_parallel = True

In [34]:
import transformers
from datetime import datetime


project = "train-wikitext"
base_model_name = "distilbert/distilgpt2"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name


tokenizer.pad_token = tokenizer.eos_token


trainer = transformers.Trainer(
    model=model,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_eval_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=10,
        gradient_accumulation_steps=4,
        max_steps=100, # before was 1000
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=500,
        fp16=True,  # before was bf16 = True
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="none",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}",          # Name of the W&B run (optional)
        # remove_unused_columns=False
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    tokenizer=tokenizer
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

TrainOutput(global_step=100, training_loss=5.028553771972656, metrics={'train_runtime': 434.2078, 'train_samples_per_second': 9.212, 'train_steps_per_second': 0.23, 'total_flos': 245197910753280.0, 'train_loss': 5.028553771972656, 'epoch': 0.0})

In [35]:
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_inputs, max_new_tokens=100, pad_token_id=2)[0], skip_special_tokens=True))

Transformers are a great way to get a better understanding of the game and how to play it.





















































































In [36]:
trainer.save_model("20240222_distilgpt2_100")